In [ ]:
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

### 1-D Gradient Checking
$J(\theta) = \theta x$

In [ ]:
def forward_1d(x, theta):
    J = np.dot(x, theta)
    return J

In [ ]:
x, theta = 2, 4
J = forward_1d(x, theta)
print ("J = " + str(J))

In [ ]:
def backward_1d(x, theta):
    d_theta = x
    return d_theta

In [ ]:
x, theta = 2, 4
dtheta = backward_1d(x, theta)
print ("dtheta = " + str(dtheta))

In [ ]:
def gradient_check_1d(x, theta, epsilon = 1e-7):
    grad = backward_1d(x, theta)
    
    grad_approx = 0.5 / epsilon * (forward_1d(x, theta+epsilon) - forward_1d(x, theta-epsilon))

    diff = np.linalg.norm(grad-grad_approx) / (np.linalg.norm(grad) + np.linalg.norm(grad_approx))

    if diff < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return diff

In [ ]:
x, theta = 2, 4
difference = gradient_check_1d(x, theta)
print("difference = " + str(difference))

### N-D Gradient Checking

In [ ]:
def forward(X, Y, params):
    m = X.shape[1]
    
    W1, W2, W3 = params["W1"], params["W2"], params["W3"]
    b1, b2, b3 = params["b1"], params["b2"], params["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)

    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)

    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    loss = -1./m * ( np.dot(np.log(A3), Y.T) + np.dot(np.log(1-A3), (1-Y).T) )
    loss = np.squeeze(loss)

    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return loss, cache

In [ ]:
def backward(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
import copy

def grad_check(parameters, gradients, X, Y, epsilon = 1e-7):

    # reshaping parameters dict into linear list
    keys = []
    lin_params = []
    for key, value in parameters.items():
        lin_params += list(value.reshape(-1))
        keys.append(key)

    # reshaping gradients dict into linear list
    lin_grads = []
    for key in keys:
        lin_grads += list(gradients["d" + key].reshape(-1))

    # empty grad approx list
    gradapprox = []

    # get position of every elemetn in paramters
    # i.e. [param_key, row, col]
    for param_key, param_value in parameters.items():
        for row in range(param_value.shape[0]):
            for col in range(param_value.shape[1]):
                
                # deep copy parameters twice
                # change element at only the position we got
                theta_plus = copy.deepcopy(parameters)
                theta_plus[param_key][row][col] += epsilon

                theta_minus = copy.deepcopy(parameters)
                theta_minus[param_key][row][col] -= epsilon

                J_plus, _ = forward(X, Y, theta_plus)
                J_minus, _ = forward(X, Y, theta_minus)

                # calculate the approx. grad for the positional element
                approx = 0.5/epsilon * (J_plus - J_minus)
                gradapprox.append(approx)

    # convert list to np array for the - operator to work
    lin_grads = np.array(lin_grads)
    gradapprox = np.array(gradapprox)
    
    # get the difference
    numerator = np.linalg.norm(lin_grads - gradapprox)
    denominator = np.linalg.norm(lin_grads) + np.linalg.norm(gradapprox)
    difference = numerator/denominator

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [ ]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward(X, Y, parameters)
gradients = backward(X, Y, cache)
difference = grad_check(parameters, gradients, X, Y)